<a href="https://colab.research.google.com/github/PrinceVerma04/RAG/blob/main/RAG_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1 \
     faiss_cpu==1.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 964.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.1 MB/s eta 0:00:00


In [15]:
from glob import glob
from langchain_community.document_loaders import PyPDFLoader

In [2]:
# Import Libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate



In [3]:
import getpass
import os

In [5]:
os.environ["GROQ_API_KEY"] = getpass.getpass()


··········


In [6]:
os.environ["HF_TOKEN"] = getpass.getpass()


··········


In [7]:

# Helper function for printing docs
def pretty_print_docs(docs):
    # Iterate through each document and format the output
    for i, d in enumerate(docs):
        print(f"{'-' * 50}\nDocument {i + 1}:")
        print(f"Content:\n{d.page_content}\n")
        print("Metadata:")
        for key, value in d.metadata.items():
            print(f"  {key}: {value}")
    print(f"{'-' * 50}")  # Final separator for clarity

# Example usage
# Assuming `docs` is a list of Document objects

In [8]:
%pip install -qU langchain-community pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires langchain-core<1.0.0,>=0.3.35, but you have langchain-core 1.0.4 which is incompatible.
langchain 0.3.19 requires langchain-text-splitters<1.0.0,>=0.3.6, but you have langchain-text-splitters 1.0.0 which is incompatible.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 1.0.4 which is incompatible.
langchain-groq 0.2.4 requires langchain-core<0.4.0,>=0.3.33

In [16]:

# ✅ Auto-load PDFs from Documents folder
pdf_folder = "/content/Documents/*.pdf"
pdf_files = glob(pdf_folder)

In [17]:

documents = []
for file in pdf_files:
    loader = PyPDFLoader(file)
    documents.extend(loader.load())

print(f"Loaded {len(pdf_files)} PDF files ✅")

Loaded 5 PDF files ✅


In [21]:
# Import necessary libraries
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# The PDF files are already loaded into the 'documents' variable from a previous cell.
# No need to load them again here with a directory path, so we remove the following lines:
# file_path = "/content/Documents"
# loader = PyPDFLoader(file_path)
# documents = loader.load()  # <-- load the PDF into documents

# Split documents into chunks of 500 characters with 100 characters overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
# Add unique IDs to each text chunk
for idx, text in enumerate(texts):
    text.metadata["id"] = idx


print(documents)

[Document(metadata={'producer': 'Acrobat Distiller 11.0 (Windows); modified using iText® Core 7.2.4 (AGPL version) ©2000-2022 iText Group NV', 'creator': 'LaTeX with hyperref package', 'creationdate': '2025-02-08T10:21:28+05:30', 'moddate': '2025-02-12T12:23:54-05:00', 'ieee article id': '10839331', 'ieee issue id': '10834412', 'subject': 'IEEE Open Journal of Signal Processing;2025;6; ;10.1109/OJSP.2025.3529377', 'ieee publication id': '8782710', 'title': 'SpoofCeleb: Speech Deepfake Detection and SASV in the Wild', 'source': '/content/Documents/SpoofCeleb_Speech_Deepfake_Detection_and_SASV_in_the_Wild (1).pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}, page_content='Received 17 September 2024; revised 3 December 2024; accepted 24 December 2024. Date of publication 13 January 2025;\ndate of current version 12 February 2025. The review of this article was arranged by Associate Editor Zafar Raﬁi.\nDigital Object Identiﬁer 10.1109/OJSP .2025.3529377\nSpoofCeleb: Speech Deepfake D

In [11]:

# Create embeddings for the text chunks
embedding = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})

# Initialize a FAISS (Vector Store) retriever with the text embeddings
retriever = FAISS.from_documents(texts, embedding).as_retriever(search_kwargs={"k": 20})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [12]:
# Import the RetrievalQA chain for question-answering tasks
from langchain.chains import RetrievalQA

# Define a query and retrieve relevant documents
query = "give me summary of this?"

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.9
)

# Create a RetrievalQA chain using the language model and the retriever
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Invoke the chain with a specific query to get a response
reponse = chain.invoke(query)

# Print the result of the response
print(reponse["result"])

**Summary of the ECAPA‑TDNN speaker‑verification architecture (as described in the excerpt)**  

---  

### 1. Goal  
Build a more powerful TDNN‑based speaker‑embedding extractor (x‑vector) that outperforms previous systems on VoxCeleb and other benchmarks.

### 2. Core Architectural Innovations  

| Innovation | What it does | Why it helps |
|-----------|--------------|--------------|
| **1‑D Res2Net blocks (SE‑Res2Blocks)** | Replace the original 1‑D Conv1D layers with a multi‑scale Res2Net module (scale = 8) that expands temporal context using larger kernels and dilation. | Captures fine‑ and coarse‑grained temporal patterns without a huge parameter increase. |
| **Squeeze‑Excitation (SE) on each block** | “Squeeze” → compute per‑channel descriptor *z* = mean of frame‑level features across time. “Excitation” → two FC layers (bottleneck) generate channel‑wise weights *s* via sigmoid. | Learns global channel inter‑dependencies, allowing the network to emphasize the most informative ch

In [23]:
query = "Explain the role fo infoedge _data_scientist"
response = chain.invoke(query)
print(response["result"])


**InfoEdge — Data Scientist Role Overview**

InfoEdge (the parent company behind platforms such as Naukri.com, 99acres, Jeevansathi, and Shiksha) relies heavily on data‑driven decision‑making to improve its products, attract and retain users, and stay ahead of the competition. A Data Scientist at InfoEdge is a key member of the analytics & machine learning (ML) team whose work spans the entire data pipeline—from raw data collection to delivering actionable insights and production‑grade models. Below is a concise description of the typical responsibilities, skills, and day‑to‑day activities you can expect in this role.

---

## 1. Core Responsibilities

| Area | What the Data Scientist Does |
|------|------------------------------|
| **Business Problem Framing** | • Work with product managers, engineers, and business stakeholders to translate high‑level goals (e.g., “increase job‑application conversion” or “improve property‑search relevance”) into concrete, measurable analytics problems